In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ner-cleaned/kaggle_cleaned.csv
/kaggle/input/news-article/cleaned_file.csv


In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import TweetTokenizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, roc_auc_score, recall_score, precision_score

In [4]:
data = pd.read_csv("/kaggle/input/news-article/cleaned_file.csv",encoding="latin")
#data['unnamed_cleaned_articles'] = data['unnamed_cleaned_articles'].astype(str)
data['article'] = data['article'].astype(str)

In [5]:
data_clean = data.copy()
sentiment_map = {'negative':0,'neutral':1,'positive':2}

data_clean['sentiments'] = data_clean['sentiments'].map(sentiment_map)

data_clean['text'] = data_clean['article'].apply(lambda x: BeautifulSoup(x, "lxml").text)
#data_clean['text'] = data_clean['unnamed_cleaned_articles'].apply(lambda x: BeautifulSoup(x, "lxml").text)

/tmp/ipykernel_33/3701256703.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  data_clean['text'] = data_clean['article'].apply(lambda x: BeautifulSoup(x, "lxml").text)


In [6]:
data_clean = data_clean.loc[:, ['text', 'sentiments']]

In [7]:
data_clean

,text,sentiments
0,Microsoft (MSFT) is still growing at a fast pa...,2
1,Microsoft-owned LinkedIn generated $1.7 billio...,2
2,Microsoft has launched a Copilot assistant for...,1
3,Microsoft's Path to Becoming the Largest Compa...,2
4,Non-deal roadshows (NDRs) allow institutional ...,2
...,...,...
9264,Oversea-Chinese Banking Corp. Limited (OCBC) (...,1
9265,"The earnings season is coming to a close, and ...",2
9266,The Monetary Authority of Singapore (MAS) has ...,1
9267,"The Straits Times Index (SGX: ^STI), or STI, i...",1


In [8]:
neutral_mapping = {0:3,1:1,2:3}
data_clean['neutral_sentiments'] = data_clean['sentiments'].map(neutral_mapping)

In [9]:
data_clean_pn = data_clean[data_clean['sentiments'] != 0]

In [10]:
train_pn, test_pn = train_test_split(data_clean_pn, test_size=0.2, random_state=42)
X_train_pn = train_pn['text'].values
X_test_pn = test_pn['text'].values
y_train_pn = train_pn['sentiments']
y_test_pn = test_pn['sentiments']

In [11]:
train_neutral, test_neutral = train_test_split(data_clean, test_size=0.2, random_state=42)
X_train_neutral = train_neutral['text'].values
X_test_neutral = test_neutral['text'].values
y_train_neutral = train_neutral['neutral_sentiments']
y_test_neutral = test_neutral['neutral_sentiments']

In [12]:


def stem(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

en_stopwords = list(stopwords.words("english")) 

vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenize,
    lowercase = True,
    ngram_range=(1, 1),
    stop_words = en_stopwords)

In [13]:
kfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [14]:
np.random.seed(42)

pipeline_svm_pn = make_pipeline(vectorizer, 
                            SVC(kernel='linear', gamma=0.5, probability=True))

grid_svm_pn = GridSearchCV(pipeline_svm_pn,
                    param_grid = {'svc__C': [ 0.1]}, 
                    cv = kfolds,
                    scoring="roc_auc",
                    verbose=1,   
                    n_jobs=-1) 

grid_svm_pn.fit(X_train_pn, y_train_pn)
grid_svm_pn.score(X_test_pn, y_test_pn)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


0.9151629212231899

In [15]:
np.random.seed(42)

pipeline_svm_neutral= make_pipeline(vectorizer, 
                            SVC(kernel='linear', gamma=0.5,probability=True))

grid_svm_neutral = GridSearchCV(pipeline_svm_neutral,
                    param_grid = {'svc__C': [ 0.1]}, 
                    cv = kfolds,
                    scoring="roc_auc",
                    verbose=1,   
                    n_jobs=-1) 

grid_svm_neutral.fit(X_train_neutral, y_train_neutral)
grid_svm_neutral.score(X_test_neutral, y_test_neutral)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


0.898178525303208

In [ ]:
import pickle

filename_neu = '/kaggle/output/neutral_model.sav'
filename_pos_neg = '/kaggle/output/pos_neg_model.sav'
pickle.dump(grid_svm_pn, open(filename_neu, 'wb'))
pickle.dump(grid_svm_nn, open(filename_pos_neg, 'wb'))